In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
import gc

# gc.collect()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
csv_types = {
    'hr': np.uint16,
    'hg': np.uint16,
    'hb': np.uint16,
    'sr': np.uint8,
    'sg': np.uint8,
    'sb': np.uint8,
}

In [ ]:
data = pd.read_csv('../data/s07e02_1.xz', dtype=csv_types)
# data = data.sample(frac=1)
# data.reset_index(drop=True, inplace=True)

# data.drop_duplicates(inplace=True)

# index = data.pop('i')

display(data)
display(data.dtypes)

In [ ]:
features = data[['hr', 'hg', 'hb']]
labels = data[['sr', 'sg', 'sb']]

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features.values, labels.values))
# dataset = tf.data.Dataset.from_tensor_slices((features, labels))

for feat, targ in dataset.take(5):
    print('Features: {}, Target: {}'.format(feat, targ))

train_dataset = dataset.shuffle(len(features) // 48).batch(2048)

In [ ]:
# free up any ram not needed for training
data = None
features = None
labels = None
dataset = None

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
  
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [house_value]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'], label='Train Error')
#   plt.plot(hist['epoch'], hist['val_mean_absolute_error'], label = 'Val Error')
#   plt.ylim([0,max(hist['val_mean_absolute_error'].max(), hist['mean_absolute_error'].max())])
    plt.ylim([0, hist['mean_absolute_error'].max()])
    plt.legend()
    
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$house_value^2$]')
    plt.plot(hist['epoch'], hist['mean_squared_error'], label='Train Error')
#   plt.plot(hist['epoch'], hist['val_mean_squared_error'], label = 'Val Error')
#   plt.ylim([0,max(hist['val_mean_squared_error'].max(), hist['mean_squared_error'].max())])
    plt.ylim([0, hist['mean_squared_error'].max()])
    plt.legend()
    plt.show()

In [ ]:
#   model = keras.Sequential([
#     layers.Dense(128, activation=tf.nn.relu, input_shape=[3]),
#     layers.Dense(256, activation=tf.nn.relu),
#     layers.Dense(128, activation=tf.nn.relu),
#     layers.Dense(16, activation=tf.nn.relu),
#     layers.Dense(3)
#   ])

#   model = keras.Sequential([
#     layers.Dense(256, activation=tf.nn.relu, input_shape=[3]),
#     layers.Dense(786, activation=tf.nn.relu),
#     layers.Dense(256, activation=tf.nn.relu),
#     layers.Dense(32, activation=tf.nn.relu),
#     layers.Dense(3)
#   ])
  

# 128 - 128 - 32 seems promising
# 64 - 64 is pretty good
# 32 - 32 works too
# 16 - 16 almost works 9:02 still blocks
# 8 - 8 yellow logo is bad. didn't test to 9:02
# 10 - 10 has a better logo, but still not great

def build_model():
    model = keras.Sequential([
        layers.Dense(24, activation=tf.nn.relu, input_shape=[3]),
        layers.Dense(24, activation=tf.nn.relu),
        layers.Dense(3)
    ])
    
    optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)
    
    # loss: mean_squared_error
    model.compile(loss='mean_absolute_error',
                  optimizer=optimizer,
                  metrics=['mean_absolute_error', 'mean_squared_error'])
    return model

In [ ]:
restore_path = None
restore_path = "../checkpoints/weights-01-3.23.hdf5"

model = build_model()
display(model.summary())

if restore_path is not None:
    model.load_weights(restore_path)
    display("loaded model from:" + restore_path)

gc.collect()

In [ ]:
display(model.summary())

# display_callback = TQDMNotebookCallback()

filepath = "../checkpoints/weights-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss')

history = model.fit(train_dataset, epochs=1, callbacks=[checkpoint])

# plot_history(history)

In [ ]:
def predict(hr, hg, hb):
    tf_in = tf.convert_to_tensor([[hr, hg, hb]])
    sdr_out = model.predict(tf_in)
    sr, sg, sb = sdr_out[0]
    sr = np.clip(sr, 0.0, 255.0)
    sg = np.clip(sg, 0.0, 255.0)
    sb = np.clip(sb, 0.0, 255.0)
    return sr, sg, sb

def batch_predict(lst):
    tf_in = tf.convert_to_tensor(lst)
    return batch_predict_tf(tf_in)

def batch_predict_tf(tf_in):
    sdr_out = model.predict(tf_in)
    return np.clip(sdr_out, 0.0, 255.0)

In [ ]:
predict(50000, 50000, 0)

In [ ]:
# usually 17, 35, or 65. 129 doesn't work in some ffmpeg builds
lut_size = 129

lut_step_size = 65535.0 / lut_size

def luti_to_hdr(i):
    return lut_step_size * i

def sdr_to_lutv(c):
    return c / 255.0

def write_lut_fast():
    lut_file = open("../generated_lut.cube", "w+")
    lut_file.write("TITLE \"HDR_2_SDR_generated_lut\"")
    lut_file.write("\n")
    lut_file.write("LUT_3D_SIZE " + str(lut_size))
    lut_file.write("\n")
    for bi in range(0, lut_size):
        for gi in range(0, lut_size):
            ril = list(range(0, lut_size))
            hdr_list = [[luti_to_hdr(ri), luti_to_hdr(gi), luti_to_hdr(bi)] for ri in ril]
            prediction_list = batch_predict(hdr_list)
            for sr, sg, sb in prediction_list:
                lr, lg, lb = sdr_to_lutv(sr), sdr_to_lutv(sg), sdr_to_lutv(sb)
                lut_file.write(f"{lr:.6f} {lg:.6f} {lb:.6f}")
                lut_file.write("\n")
    lut_file.close()

In [ ]:
write_lut_fast()